In [11]:
import requests
from PIL import Image
from io import BytesIO
from diffusers import LDMSuperResolutionPipeline
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "CompVis/ldm-super-resolution-4x-openimages"

# load model and scheduler
pipeline = LDMSuperResolutionPipeline.from_pretrained(model_id)
pipeline = pipeline.to(device)

# let's download an  image
url = "https://user-images.githubusercontent.com/38061659/199705896-b48e17b8-b231-47cd-a270-4ffa5a93fa3e.png"


response = requests.get(url)

ground = Image.open(
    "/home/kareem/hacking/research/AI_Love/denoising/Simulated_data_sennd/Noise_free/image_0102.jpg"
)
# low_res_img = Image.open(BytesIO(response.content)).convert("RGB")
low_res_img = Image.open(
    "/home/kareem/hacking/research/AI_Love/denoising/CNN_TASK/SA&SK/image_0102.jpg"
)
ground= ground.resize((224,224))
low_res_img = low_res_img.resize((224, 224))

# run pipeline in inference (sample random noise and denoise)
upscaled_image = pipeline(low_res_img, num_inference_steps=20, eta=1).images[0]
# save image
upscaled_image.save("ldm_generated_image.png")

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/kareem/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/kareem/.cache/huggingface/hub/models--CompVis--ldm-super-resolution-4x-openimages/snapshots/0b55ddf931a8e3a1b426b3a50ddcf325ff84f668/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/home/kareem/mambaforge/envs/dinov2/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
An error occurred while trying to fetch /home/kareem/.cache/huggingface/hub/models--CompV

  0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
# from torch import cuda
# cuda.empty_cache()

In [13]:
# x = upscaled_image.resize((128,128))
x = upscaled_image.resize((224,224))

In [14]:
y = low_res_img

In [17]:
from torchvision import transforms
transform = transforms.Compose(
    [transforms.ToTensor()]  # Converts the image to a PyTorch tensor
)

# Apply the transformation
x_t= transform(x)
y_t = transform(y)
ground = transform(ground)
# Optionally, add a batch dimension
# img_tensor = img_tensor.unsqueeze(0)
x_t.shape,y_t.shape,ground.shape

(torch.Size([3, 224, 224]),
 torch.Size([3, 224, 224]),
 torch.Size([3, 224, 224]))

In [20]:
from torchmetrics.functional.image import peak_signal_noise_ratio

peak_signal_noise_ratio(x_t, y_t)

tensor(28.9747)

In [21]:
peak_signal_noise_ratio(x_t,ground)

tensor(27.2647)

In [22]:
peak_signal_noise_ratio(ground,y_t)

tensor(26.6704)

In [23]:
peak_signal_noise_ratio(ground,x_t)

tensor(27.2647)